In [1]:
#!pip install pandas numpy emoji langdetect transformers torch


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from scipy.special import softmax
import urllib.request, csv

c:\Users\HRMic\OneDrive - ADvTECH Ltd\Documents\VS code\Python_VS\TWITTER\Research\gpuEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv('classified samples/2024_Classified_Balanced.csv')
df.info()

,label,tweet
0,Left,@ USER Fuck Joe Biden fuck Ukraine fuck the De...
1,Right,Fly them back to their home country just like ...
2,Right,Wow ! I ca n't believe the @ USER . They 're s...
3,Right,@ USER @ USER @ USER The election be rig again...
4,Right,@ USER @ USER @ USER @ USER @ USER Why be acto...


In [ ]:
# preprocess text (username and link replacement)
# to follow consistency with the model's training data
def preprocess(text):
    new_text = []
    for t in str(text).lower().split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
#(Hanu, 2020; Lai-Lopez et al., 2025)

In [18]:
# Load model and tokenizer
MODEL = 'textdetox/twitter-xlmr-toxicity-classifier' # specify the model name based on the task
# Load tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, use_safetensors=True)
labels = ["non-toxic", "toxic"]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Model loaded to device:", device)
#(Hanu, 2020; Lai-Lopez et al., 2025)

Model loaded to device: cuda


In [23]:

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))

CUDA available: True
Number of GPUs: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [24]:
texts = df["tweet"].astype(str).apply(preprocess).tolist()


In [25]:
batch_size = 64  
all_labels = []
all_probs = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]

    encoded = tokenizer(
        batch,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        output = model(**encoded)
        logits = output.logits.detach().cpu().numpy()
        probs = softmax(logits, axis=1)   # vectorized softmax

    pred_ids = np.argmax(probs, axis=1)

    # store
    all_labels.extend([labels[j] for j in pred_ids])
    all_probs.extend(probs)


In [26]:
df["toxicity_label"] = all_labels
df_probs = pd.DataFrame(all_probs, columns=labels)
df = pd.concat([df, df_probs], axis=1)


In [ ]:
if "label" in df.columns:
    avg_tox = df.groupby("label")[labels].mean().reset_index()
    print("\nAverage Toxicity Probabilities by Political Leaning:")
    print(avg_tox)

df.to_csv("analysis/2024_toxicity_results.csv", index=False)
#(Hanu, 2020; Lai-Lopez et al., 2025)


Average Toxicity Probabilities by Political Leaning:
   label  non-toxic     toxic
0   Left   0.663978  0.336022
1  Right   0.639528  0.360472


In [ ]:
df = pd.read_csv("analysis/2024_toxicity_results.csv")
df['toxicity_probs'] = df[['non-toxic', 'toxic']].apply(
    lambda x: np.array(x, dtype=float),
    axis=1
)
df.to_csv("analysis/2024_toxicity_results.csv", index=False)